In [16]:
from aif360.metrics.classification_metric import ClassificationMetric
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score , classification_report, confusion_matrix, roc_auc_score,mean_squared_error,f1_score 
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.datasets import StandardDataset , BinaryLabelDataset
from sklearn.tree import  DecisionTreeClassifier 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
import seaborn as sns
import joblib
from utils import make_dataset, display_results
from sklearn.preprocessing import MinMaxScaler 
MM= MinMaxScaler()


# Original Dataset

In [17]:
#Testing Unfair Model Using TrainTest split 1
TrainData1= pd.read_csv(r"C:\Users\dehob001\OneDrive - University of South Australia(1)\Python Projects\Survey of Fairness "
                        r"Algorithms\Baselines\German Credit DataSet\5Fold German Splits\Shuffle 1\Train\Train 5.csv")
TestData1= pd.read_csv(r"C:\Users\dehob001\OneDrive - University of South Australia(1)\Python Projects\Survey of Fairness "
                       r"Algorithms\Baselines\German Credit DataSet\5Fold German Splits\Shuffle 1\Test\Test 5.csv")

#Trainset
Train1= TrainData1.drop('default', axis= 1)
TrainLabel1= TrainData1 ['default']
Fitter1= MM.fit(Train1)
Transformed_Train1= Fitter1.transform(Train1) 
Train1DF = pd.DataFrame(Transformed_Train1  , columns= Train1.columns)

#TestSEt
Test1= TestData1 .drop('default', axis= 1)
TestLabel1= TestData1['default']
Transformed_Test1= Fitter1.transform(Test1)
Test1DF=pd.DataFrame(Transformed_Test1 , columns= Test1.columns)

TrainTest= pd.concat([TrainData1, TestData1 ])

# print(Train1DF.shape, Test1DF.shape, TrainLabel1.shape, TestLabel1.shape )
# print(TrainData1 )

# DI proportion calculator

In [18]:
# #proportion calculator
# # def calc_prop(data, group_col, group, output_col, output_val):
# #     new = data[data[group_col] == group]
# #     return len(new[new[output_col] == output_val])/len(new)

# Setting  Train and Test into BinaryLabel

In [19]:
#Transforming the Train and Test Set to BinaryLabel
class Train(StandardDataset):
    def __init__(self,label_name= 'default',
                 favorable_classes= [1],protected_attribute_names=['age'],   privileged_classes=[[1]], ):


        super(Train, self).__init__(df=TrainData1  , label_name=label_name ,
            favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
            privileged_classes=privileged_classes ,
           )



advantagedGroup= [{'age':1}]
disadvantagedGroup= [{'age':0}]

BLD_Train= Train(protected_attribute_names= ['age'],
                       privileged_classes= [[1]])



class Test(StandardDataset):
    def __init__(self,label_name= 'default',
                 favorable_classes= [1],protected_attribute_names=['age'],   privileged_classes=[[1]], ):


        super(Test, self).__init__(df=TestData1  , label_name=label_name ,
            favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
            privileged_classes=privileged_classes ,
           )



advantagedGroup= [{'age':1}]
disadvantagedGroup= [{'age':0}]

BLD_Test= Test(protected_attribute_names= ['age'],
                       privileged_classes= [[1]])

# BLD_Train 

# Checking bias in Raw Data (Disparate Impact, Statistical Parity and consistency)

In [20]:

# You dont need to do this
DataBias_Checker = BinaryLabelDatasetMetric(BLD_Test    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)

dsp= DataBias_Checker .statistical_parity_difference() 
dif= DataBias_Checker.consistency()  
ddi= DataBias_Checker.disparate_impact() 

print('The Statistical Parity diference is = {diff}'.format(diff=  dsp  ))
print('Individual Fairness is = {IF}'.format( IF=  dif ))
print('Disparate Impact is = {IF}'.format( IF=   ddi  ))





The Statistical Parity diference is = 0.07557268250909865
Individual Fairness is = [0.669]
Disparate Impact is = 1.2563543936092954


# Checking Accuracy and Predictive Measures

### Logistic Regression

In [21]:
#Checking Prior Biases
Clf= LogisticRegression( )
Clf .fit(Train1DF, TrainLabel1)


acc= accuracy_score( TestLabel1 , Clf .predict(Test1DF))
roc= roc_auc_score(TestLabel1 , Clf.predict(Test1DF ))
f1=f1_score(TestLabel1 ,Clf.predict(Test1DF))
rmse= mean_squared_error(TestLabel1,Clf.predict(Test1DF), squared= True)

print(acc)
print( roc)
print(f1)
print(rmse)

0.755
0.6811534379054134
0.5504587155963302
0.245


 # Checking Discrimniation of Biased Classifier

In [22]:


# predictedDF= pd.DataFrame({'Age': Test1DF['age'],'predictions': Predictions })
# #print(predictedDF)

# #Calculate DI of classifications
# LR_upriv= calc_prop(predictedDF ,'Age',0,'predictions',1)
# LR_priv= calc_prop(predictedDF ,'Age',1,'predictions',1)
# print('Favourable outcome for unprivilege group:',LR_upriv )
# print('Favourable outcome for privilege group: ',LR_priv )
# print('DI: ', LR_upriv/LR_priv )

In [23]:
Predictions = Clf.predict(Test1DF)
PredictedDataSet= TestData1.copy(deep=True)
PredictedDataSet['default'] = Predictions

class PredTest(StandardDataset):
    def __init__(self,label_name= 'default',
                 favorable_classes= [1],protected_attribute_names=['age'],   privileged_classes=[[1]], ):


        super(PredTest, self).__init__(df=PredictedDataSet  , label_name=label_name ,
            favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
            privileged_classes=privileged_classes ,
           )



advantagedGroup= [{'age':1}]
disadvantagedGroup= [{'age':0}]



BLD_PredTest= PredTest(protected_attribute_names= ['age'],
                       privileged_classes= [[1]])




In [24]:

OldOdf= pd.read_csv(r"C:\Users\dehob001\OneDrive - University of South Australia(1)\Python Projects\Survey of Fairness Algorithms\Disparate Impact Remover\Odf.csv")

ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)

SP=ClassifierBias .statistical_parity_difference() 
IF=ClassifierBias.consistency()
DI=ClassifierBias.disparate_impact()
EOP=ClassifierBias.true_positive_rate_difference()
EO=ClassifierBias.average_odds_difference()
FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
NPV=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
PPV=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)

BGE = ClassifierBias.between_group_generalized_entropy_index()
WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
BGTI = ClassifierBias.between_group_theil_index()
WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index() 

print('The Statistical Parity diference is = {diff}'.format(diff=  SP ))
print('Consistency is = {IF}'.format( IF= IF))
print('Disparate Impact is = {IF}'.format( IF= DI))
print('Equality of Opportunity = {IF}'.format( IF=EOP ))
print('Equalized Odds = {IF}'.format( IF= EO))
print('Positive Predictive Value = {IF}'.format( IF= PPV))
print('False Discovery Rate Difference = {IF}'.format( IF= FDR ))
print('Negative Predictive Value Difference = {IF}'.format( IF=NPV  ))
print('False Omission Rate Difference = {IF}'.format( IF= FOR))
print('Between-Group Entropy Index= {IF}'.format( IF= BGE ))
print('Within-Group Entropy Index = {IF}'.format( IF= WGE ))
print('Between-Group Theil Index = {IF}'.format( IF= BGTI))
print('Within-Group Theil Index= {IF}'.format( IF= WGTI ))


Odf= pd.DataFrame({'acc':acc,'roc':roc, 'f1':f1,'rmse':rmse,'SP':SP,'IF':IF,'DI':DI, 'EOP':EOP,'EO': EO, 'PPV':PPV,'FDR':FDR, 'NPV': NPV,'FOR':FOR,'BGE':BGE , 'WGE':WGE, 'BGTI':BGTI,'WGTI':WGTI})

OldOdf= pd.concat([OldOdf, Odf ])
OldOdf.to_csv('Odf.csv')









The Statistical Parity diference is = 0.06508242346392634
Consistency is = [0.669]
Disparate Impact is = 1.2814814814814814
Equality of Opportunity = 0.009803921568627472
Equalized Odds = 0.03166184506589523
Positive Predictive Value = 0.0
False Discovery Rate Difference = 0.0
Negative Predictive Value Difference = -0.06766917293233088
False Omission Rate Difference = 0.06766917293233082
Between-Group Entropy Index= 7.349691058246566e-06
Within-Group Entropy Index = 0.13770033425758177
Between-Group Theil Index = 7.369857101799226e-06
Within-Group Theil Index= 0.20064147876767444


# Repairing Biased Data Using Disparate Impact Remover

In [25]:
DIR = DisparateImpactRemover(repair_level=1)

DI_Train = DIR.fit_transform(BLD_Train )
DI_Test = DIR.fit_transform(BLD_Test)
DI_Test 


               instance weights features                \
                                                         
                                   month credit_amount   
instance names                                           
0                           1.0     21.0        1216.0   
1                           1.0     18.0        1366.0   
2                           1.0     18.0        4272.0   
3                           1.0     11.0         652.0   
4                           1.0     11.0        7297.0   
...                         ...      ...           ...   
195                         1.0     11.0        1216.0   
196                         1.0     27.0        2221.0   
197                         1.0     11.0         601.0   
198                         1.0     45.0        1845.0   
199                         1.0     45.0        3016.0   

                                                                     \
                                                          

# Repaired Data 

In [26]:
RepairedTrain=  pd.DataFrame(np.hstack([DI_Train .labels,DI_Train .features]),columns=TrainData1.columns)
RepairedTest=  pd.DataFrame(np.hstack([DI_Test .labels,DI_Test .features]),columns=TestData1.columns)
TotalRepairedDF= pd.concat([RepairedTrain ,RepairedTest ])
#TotalRepairedDF.to_csv('repaired.csv')


# Checking bias in Repaired Data (Disparate Impact, Statistical Parity and consistency)

In [27]:
Olddb= pd.read_csv(r"C:\Users\dehob001\OneDrive - University of South Australia(1)\Python Projects\Survey of Fairness Algorithms\Disparate Impact Remover\db.csv")

FairDataBias_Checker = BinaryLabelDatasetMetric(DI_Test    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
fdi= FairDataBias_Checker.disparate_impact()  
fsp=FairDataBias_Checker .statistical_parity_difference()
fif= FairDataBias_Checker.consistency() 

print('Disparate Impact is = {IF}'.format( IF=  fdi  ))
print('The Statistical Parity diference is = {diff}'.format(diff= fsp   ))
print('Individual Fairness is = {IF}'.format( IF= fif  ))

DataBias= pd.DataFrame({'DI':ddi, 'SP':dsp, 'IF':dif,'FDI':fdi,'FSP':fsp, 'FIF':fif})
Olddb= pd.concat([Olddb, DataBias ])
Olddb .to_csv('db.csv')




Disparate Impact is = 1.2563543936092954
The Statistical Parity diference is = 0.07557268250909865
Individual Fairness is = [0.673]


# Setting up  Repaired TrainTest for Classification

In [28]:
Trainer= RepairedTrain .drop('default', axis= 1)
TrainLabel= RepairedTrain ['default']
Fitter= MM.fit(Trainer)
Transformed_Train= Fitter.transform(Trainer) 
TrainDF = pd.DataFrame(Transformed_Train , columns= Trainer.columns)

#TestSEt
Tester= RepairedTest.drop('default', axis= 1)
TestLabel= RepairedTest ['default']
Transformed_Test= Fitter.transform(Tester)
TestDF=pd.DataFrame(Transformed_Test , columns= Tester.columns)

# Logistic Regression  Repaired Data

In [29]:
#Checking Fair metrics
Clfr= LogisticRegression( )
Clfr.fit(TrainDF, TrainLabel)

acc= accuracy_score( TestLabel , Clfr .predict(TestDF))
roc= roc_auc_score(TestLabel , Clfr.predict(TestDF ))
f1=f1_score(TestLabel ,Clfr.predict(TestDF))
rmse= mean_squared_error(TestLabel,Clfr.predict(TestDF), squared= True)

print(acc)
print( roc)
print(f1)
print(rmse)


0.765
0.6837480835004128
0.5523809523809523
0.235


# Setting Fair Predictions to BLD for analysis

In [30]:
Predicton = Clfr.predict(TestDF)
FairPredictedData= TestData1.copy(deep=True)
FairPredictedData['default'] = Predicton

class FairPredTest(StandardDataset):
    def __init__(self,label_name= 'default',
                 favorable_classes= [1],protected_attribute_names=['age'],   privileged_classes=[[1]], ):


        super(FairPredTest, self).__init__(df=FairPredictedData  , label_name=label_name ,
            favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
            privileged_classes=privileged_classes ,
           )



advantagedGroup= [{'age':1}]
disadvantagedGroup= [{'age':0}]



BLD_FairPredTest= FairPredTest(protected_attribute_names= ['age'],
                       privileged_classes= [[1]])




# Checking For Bias in Fair Predictions

In [35]:
Olddf= pd.read_csv(r"C:\Users\dehob001\OneDrive - University of South Australia(1)\Python Projects\Survey "
                   r"of Fairness Algorithms\Disparate Impact Remover\df.csv")
FairClassifierBias = ClassificationMetric( BLD_Test,BLD_FairPredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
SP=FairClassifierBias .statistical_parity_difference() 
IF=FairClassifierBias.consistency()
DI=FairClassifierBias.disparate_impact()
EOP=FairClassifierBias.true_positive_rate_difference()
EO=FairClassifierBias.average_odds_difference()
FDR= FairClassifierBias.false_discovery_rate(privileged=False)- FairClassifierBias.false_discovery_rate(privileged=True)
NPV=FairClassifierBias.negative_predictive_value(privileged=False)-FairClassifierBias.negative_predictive_value(privileged=True)
FOR=FairClassifierBias.false_omission_rate(privileged=False)-FairClassifierBias.false_omission_rate(privileged=True)
PPV=FairClassifierBias.positive_predictive_value(privileged=False) -FairClassifierBias.positive_predictive_value(privileged=True)

BGE = FairClassifierBias.between_group_generalized_entropy_index()
WGE = FairClassifierBias.generalized_entropy_index()-FairClassifierBias.between_group_generalized_entropy_index()
BGTI = FairClassifierBias.between_group_theil_index()
WGTI = FairClassifierBias.theil_index() -FairClassifierBias.between_group_theil_index() 

print('The Statistical Parity diference is = {diff}'.format(diff=  SP ))
print('Consistency is = {IF}'.format( IF= IF))
print('Disparate Impact is = {IF}'.format( IF= DI))
print('Equality of Opportunity = {IF}'.format( IF=EOP ))
print('Equalized Odds = {IF}'.format( IF= EO))
print('Positive Predictive Value = {IF}'.format( IF= PPV))
print('False Discovery Rate Difference = {IF}'.format( IF= FDR ))
print('Negative Predictive Value Difference = {IF}'.format( IF=NPV  ))
print('False Omission Rate Difference = {IF}'.format( IF= FOR))
print('Between-Group Entropy Index= {IF}'.format( IF= BGE ))
print('Within-Group Entropy Index = {IF}'.format( IF= WGE ))
print('Between-Group Theil Index = {IF}'.format( IF= BGTI))
print('Within-Group Theil Index= {IF}'.format( IF= WGTI ))
df=pd.DataFrame({'acc':acc,'roc':roc, 'f1':f1,'rmse':rmse,'SP':SP,'IF':IF,'DI':DI, 'EOP':EOP,'EO': EO, 'PPV':PPV,'FDR':FDR, 'NPV': NPV,'FOR':FOR,'BGE':BGE , 'WGE':WGE, 'BGTI':BGTI,'WGTI':WGTI})

Olddf= pd.concat([Olddf,df])
Olddf.to_csv('df.csv') 



The Statistical Parity diference is = 0.08820381074716332
Consistency is = [0.669]
Disparate Impact is = 1.4238683127572016
Equality of Opportunity = 0.02941176470588236
Equalized Odds = 0.053760848601735786
Positive Predictive Value = -0.04166666666666663
False Discovery Rate Difference = 0.041666666666666685
Negative Predictive Value Difference = -0.06607760276603925
False Omission Rate Difference = 0.06607760276603916
Between-Group Entropy Index= 1.1126587327766335e-05
Within-Group Entropy Index = 0.136018614521693
Between-Group Theil Index = 1.1089439995285496e-05
Within-Group Theil Index= 0.20245080960431627
